In [7]:
import pickle
import numpy as np
import scipy as sc
import pandas as pd
from sklearn.feature_selection import *
from sklearn.linear_model import *
from sklearn.metrics import *
from sklearn.cross_validation import *
from sklearn.preprocessing import *

In [8]:
data = pd.read_csv("./Data/DotaTrainingData.txt", sep = "[\n]|[,]", names = [str(x) for x in range(11)])

# Determining ratio of test and train datasets
testSize, randomState = 0 , 1

data_train, data_test, outcome_train, outcome_test = train_test_split(data.drop('10', axis=1), 
                                                                      data.iloc[:]['10'], 
                                                                      test_size = testSize, 
                                                                      random_state = randomState)

/home/luxxon/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


In [9]:
df1 = pd.get_dummies(data_train.iloc[:,:5])
df2 = pd.get_dummies(data_train.iloc[:,5:])

df3 = pd.get_dummies(data_test.iloc[:,:5])
df4 = pd.get_dummies(data_test.iloc[:,5:])

df1.columns = df1.columns.map(lambda x: x[2:])
df2.columns = df2.columns.map(lambda x: x[2:])
df3.columns = df3.columns.map(lambda x: x[2:])
df4.columns = df4.columns.map(lambda x: x[2:])

featNum = np.unique(data_train).size

df1 = df1.groupby(df1.columns, axis = 1).sum()
df2 = df2.groupby(df2.columns, axis = 1).sum()
df3 = df3.groupby(df3.columns, axis = 1).sum()
df4 = df4.groupby(df4.columns, axis = 1).sum()


features = pd.concat([df1, df2], axis=1)
features_test = pd.concat([df3, df4], axis=1)

print features.iloc[:, 0:97].shape
print features.iloc[:, 97:].shape

features = features.iloc[:, 0:97] - features.iloc[:, 97:]
features_test = features_test.iloc[:, 0:97] - features_test.iloc[:, 97:]

print features.shape
print features_test.shape

(15000, 97)
(15000, 97)
(15000, 97)
(0, 0)


In [ ]:
logicv = LogisticRegressionCV(Cs=[0.08], fit_intercept=True, cv=100, penalty='l2', n_jobs=-1,
                            solver='newton-cg', tol=0.0001, intercept_scaling=1.0, random_state = 1)

polyF = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
new_features = features.values.copy()

polyF.fit(features)

new_features[new_features == -1] = 2
new_features[new_features == 1] = 3

# Creating same team pairs feature
team_interactions = polyF.transform(new_features)

team_interactions[team_interactions == 4] = -1
team_interactions[team_interactions == 2] = 0
team_interactions[team_interactions == 3] = 0
team_interactions[team_interactions == 6] = 0
team_interactions[team_interactions == 9] = 1

team_interactions = team_interactions[:, 97:]

print "Team interactions - min/max, shape"
print np.max(team_interactions), np.min(team_interactions), team_interactions.shape

# Creating counter team pairs
new_features = np.multiply(
    polyF.transform(new_features),
    np.take(new_features, np.argmax(polyF.powers_, axis = 1), axis = 1)
)

counter_interactions = np.zeros_like(new_features)

counter_interactions[new_features == 12] = -1
counter_interactions[new_features == 18] = 1

counter_interactions = counter_interactions[:, 97:] 

print "\nCounter interactions - min/max, shape"
print np.max(counter_interactions), np.min(counter_interactions), counter_interactions.shape

# Selecting k best interaction by univariate correlation
sK = SelectKBest(f_classif, k=10)
sK.fit(team_interactions, outcome_train)

print "\nSelected team interactions: "
print [sK.get_support(True)]

team_indices = sK.get_support(True)

# Selecting k best interaction by univariate correlation
sK = SelectKBest(f_classif, k=10)
sK.fit(counter_interactions, outcome_train)

print "\nSelected counter interactions: "
print [sK.get_support(True)]

counter_indices = sK.get_support(True)

final_features = np.hstack((
        features,
        np.take(team_interactions, team_indices, axis = 1),
        np.take(counter_interactions, counter_indices, axis = 1)
    ))

print "\nFinal features shape: ", final_features.shape

logicv.fit(final_features, outcome_train.subtract(1))

print "Train accuracy achieved: ", logicv.score(final_features, outcome_train.subtract(1))


Team interactions - min/max, shape
1.0 -1.0 (15000, 4656)

Counter interactions - min/max, shape
1.0 -1.0 (15000, 4656)

Selected team interactions: 
[array([1360, 2459, 2507, 2595, 2658, 2663, 2900, 3442, 3463, 3670])]

Selected counter interactions: 
[array([ 211, 2466, 2471, 2904, 3343, 3530, 3669, 3680, 3686, 3706])]

Final features shape:  (15000, 117)


In [ ]:
# Serialising model parameters for submission
pickle.dumps([logicv.C_, logicv.Cs_, logicv.coef_, logicv.intercept_, logicv.classes_]).encode("base64")